In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib


datos = pd.read_csv(
    "C:/Users/ASUS/OneDrive/Cosas/Desktop/Escritorio Trabajo/Prototipo MI/Datos/otros-delitos.json",
    delimiter=";",  
    encoding="utf-8-sig"  
)


try:
    datos['FECHA'] = pd.to_datetime(datos['FECHA'], errors='coerce')  
    datos.dropna(subset=['FECHA'], inplace=True)  
except Exception as e:
    print(f"Error al procesar las fechas: {e}")
    exit()


datos['DIA_SEMANA'] = datos['FECHA'].dt.day_name() 
datos['HORA'] = datos['FECHA'].dt.hour 
datos['AÑO'] = datos['FECHA'].dt.year  
datos['MES'] = datos['FECHA'].dt.month_name()  

datos.dropna(subset=['BARRIO_MONTEVIDEO', 'DELITO'], inplace=True)


datos_agrupados = datos.groupby(['BARRIO_MONTEVIDEO', 'DELITO', 'AÑO', 'MES', 'DIA_SEMANA', 'HORA']).size().reset_index(name='CONTEO')


print("Convirtiendo columnas categóricas a números...")

datos_agrupados['MES'] = pd.to_datetime(datos_agrupados['MES'], format='%B').dt.month 
datos_agrupados['DIA_SEMANA'] = pd.to_datetime(datos_agrupados['DIA_SEMANA'], format='%A').dt.weekday  


print("Entrenando modelos por barrio y delito...")
modelos_barrio_delito = {}


for barrio in datos_agrupados['BARRIO_MONTEVIDEO'].unique():
    for delito in datos_agrupados['DELITO'].unique():
        try:
            # print(f"Entrenando modelo para {barrio} - {delito}...")

            
            datos_filtrados = datos_agrupados[
                (datos_agrupados['BARRIO_MONTEVIDEO'] == barrio) & 
                (datos_agrupados['DELITO'] == delito)
            ]

            
            if datos_filtrados.shape[0] < 5:
                print(f"No hay suficientes datos para {barrio} - {delito}.")
                continue

            
            X = datos_filtrados[['AÑO', 'MES', 'DIA_SEMANA', 'HORA']]
            y = datos_filtrados['CONTEO']

           
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
            rf_model.fit(X_train, y_train)

            modelos_barrio_delito[(barrio, delito)] = rf_model

        except ValueError as e:
            print(f"Error al entrenar el modelo para {barrio} - {delito}: {e}")
            continue

print("Guardando modelos...")

try:
    joblib.dump(modelos_barrio_delito, 'modelos_barrio_delito.pkl')
    print("Modelos guardados exitosamente.")
except Exception as e:
    print(f"Error al guardar los archivos: {e}")

In [2]:
# DEMO DE VERIFICACIÓN

modelos_barrio_delito = joblib.load('modelos_barrio_delito.pkl')

def convertir_a_caracteristicas(fecha_hora):
    fecha_hora = pd.to_datetime(fecha_hora)
    
    año = fecha_hora.year
    mes = fecha_hora.month
    dia_semana = fecha_hora.weekday()  # Lunes = 0 y Domingo = 6
    hora = fecha_hora.hour
    
    return [año, mes, dia_semana, hora]

fecha_hora_evaluacion = "2025-01-23 15:00:00"

caracteristicas = convertir_a_caracteristicas(fecha_hora_evaluacion)

barrio_evaluacion = "POCITOS" 
delito_evaluacion = "VIOLENCIA DOMÉSTICA" 


if (barrio_evaluacion, delito_evaluacion) in modelos_barrio_delito:
    modelo = modelos_barrio_delito[(barrio_evaluacion, delito_evaluacion)]
    prediccion = modelo.predict([caracteristicas])
    print(f"Predicción para {barrio_evaluacion} - {delito_evaluacion} a las {fecha_hora_evaluacion}: {prediccion[0]} delitos")
else:
    print(f"No hay un modelo disponible para la combinación de {barrio_evaluacion} - {delito_evaluacion}")

NameError: name 'joblib' is not defined

Ejemplo del print: Predicción para POCITOS - VIOLENCIA DOMÉSTICA a las 2025-01-23 15:00:00: 1.978047619047619 delitos

In [ ]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import joblib
import random

# CARGA Y PRUEBA 2 DEL MODELO MEJOR EJECUTADO
modelos_barrio_delito = joblib.load('modelos_barrio_delito.pkl')

def convertir_a_caracteristicas(fecha_hora):
    fecha_hora = pd.to_datetime(fecha_hora)
    return [fecha_hora.year, fecha_hora.month, fecha_hora.weekday(), fecha_hora.hour]

# Fecha y hora de la prueba
fecha_hora_evaluacion = "2025-01-23 15:00:00"
caracteristicas = convertir_a_caracteristicas(fecha_hora_evaluacion)

# Coordenadas
barrios_coordenadas = {
    'POCITOS': (-34.9158, -56.1531), 
    'ITUZAINGO': (-34.8608, -56.1598), 
    'LA TEJA': (-34.8775, -56.2301),
    'COLON CENTRO Y NOROESTE': (-34.8113, -56.2475), 
    'AIRES PUROS': (-34.8614, -56.2138), 
    'CAPURRO, BELLA VISTA': (-34.8850, -56.2218),
    'TRES CRUCES': (-34.8925, -56.1708), 
    'CENTRO': (-34.9078, -56.1919), 
    'LA COMERCIAL': (-34.8833, -56.1644),
    'PQUE. BATLLE, V. DOLORES': (-34.8917, -56.1536),
    'PUNTA GORDA': (-34.9164, -56.0694), 
    'JARDINES DEL HIPODROMO': (-34.8486, -56.1117), 
    'LAS CANTERAS': (-34.8283, -56.1953),
    'MALVIN NORTE': (-34.8806, -56.1233), 
    'SAYAGO': (-34.8283, -56.2139), 
    'BUCEO': (-34.9014, -56.1289),
    'BAÑADOS DE CARRASCO': (-34.8428, -56.0617), 
    'PUNTA CARRETAS': (-34.9275, -56.1600), 
    'MERCADO MODELO, BOLIVAR': (-34.8556, -56.1911),
    'UNION': (-34.8781, -56.1297), 
    'CORDON': (-34.9027, -56.1772), 
    'VILLA GARCIA, MANGA RURAL': (-34.7800, -56.1350),
    'PRADO, NUEVA SAVONA': (-34.8461, -56.2256), 
    'NUEVO PARIS': (-34.8139, -56.2397), 
    'CERRITO': (-34.8447, -56.1875),
    'CERRO': (-34.8897, -56.2667), 
    'PTA. RIELES, BELLA ITALIA': (-34.8372, -56.1228), 
    'PASO DE LA ARENA': (-34.8772, -56.2725),
    'BELVEDERE': (-34.8475, -56.2408), 
    'LA PALOMA, TOMKINSON': (-34.8489, -56.2203), 
    'CASAVALLE': (-34.8311, -56.1703),
    'VILLA ESPAÑOLA': (-34.8594, -56.1514), 
    'TRES OMBUES, VICTORIA': (-34.8719, -56.2597), 
    'CASABO, PAJAS BLANCAS': (-34.8614, -56.2828),
    'MALVIN': (-34.8986, -56.1186), 
    'MANGA': (-34.8214, -56.0989), 
    'CARRASCO NORTE': (-34.8869, -56.0689),
    'JACINTO VERA': (-34.8856, -56.1744), 
    'LA BLANQUEADA': (-34.8836, -56.1600), 
    'SIN CLASIFICAR': (-34.8944, -56.1886),
    'LEZICA, MELILLA': (-34.7856, -56.2567), 
    'PIEDRAS BLANCAS': (-34.8286, -56.1203), 
    'LARRAÑAGA': (-34.8839, -56.1558),
    'COLON SURESTE, ABAYUBA': (-34.8036, -56.1769), 
    'CASTRO, P. CASTELLANOS': (-34.8492, -56.1428), 
    'FLOR DE MAROÑAS': (-34.8617, -56.1283),
    'PALERMO': (-34.9094, -56.1917), 
    'ATAHUALPA': (-34.8603, -56.2003), 
    'PASO DE LAS DURANAS': (-34.8506, -56.2097),
    'PEÑAROL, LAVALLEJA': (-34.8408, -56.2211), 
    'LAS ACACIAS': (-34.8567, -56.1311), 
    'MANGA, TOLEDO CHICO': (-34.8033, -56.1028),
    'AGUADA': (-34.8839, -56.1896), 
    'CIUDAD VIEJA': (-34.9072, -56.2117), 
    'BARRIO SUR': (-34.9100, -56.1914),
    'CONCILIACION': (-34.8364, -56.2356), 
    'REDUCTO': (-34.8764, -56.1856), 
    'VILLA MUÑOZ, RETIRO': (-34.8842, -56.1806),
    'PARQUE RODO': (-34.9192, -56.1628), 
    'BRAZO ORIENTAL': (-34.8739, -56.1892), 
    'CARRASCO': (-34.8925, -56.0506),
    'MAROÑAS, PARQUE GUARANI': (-34.8581, -56.1269), 
    'LA FIGURITA': (-34.8781, -56.1811)
}


delitos_colores = {
    "HURTO": "#003366",
    "RAPIÑA": "#0066cc"
}


barrios = list(barrios_coordenadas.keys())
delitos = list(delitos_colores.keys())
predicciones_df = pd.DataFrame(index=barrios, columns=delitos)


for barrio in barrios:
    for delito in delitos:
        if (barrio, delito) in modelos_barrio_delito:
            modelo = modelos_barrio_delito[(barrio, delito)]
            prediccion = modelo.predict([caracteristicas])
            predicciones_df.loc[barrio, delito] = round(prediccion[0])
        else:
            predicciones_df.loc[barrio, delito] = 0  


mapa = folium.Map(location=[-34.9011, -56.1645], zoom_start=12, tiles='CartoDB positron')

cluster = MarkerCluster().add_to(mapa)


for barrio, (lat, lon) in barrios_coordenadas.items():
    for delito in ["HURTO", "RAPIÑA"]:  # Solo HURTO y RAPIÑA PARA EL EJMEPLO
        color = delitos_colores[delito]
        cantidad = int(predicciones_df.loc[barrio, delito])
        if cantidad > 0:  
            for _ in range(cantidad): 
                lat_offset = random.uniform(-0.002, 0.002)  
                lon_offset = random.uniform(-0.002, 0.002)
                folium.CircleMarker(
                    location=(lat + lat_offset, lon + lon_offset),
                    radius=5, 
                    color=color,
                    fill=True,
                    fill_opacity=0.7,
                    popup=f"{barrio} - {delito}: {cantidad} casos"
                ).add_to(cluster)


mapa.save("Resultados/Mapa_Delitos_HURTO_RAPIÑA_Montevideo.html")

print("Mapa generado y guardado como 'Mapa_Delitos_HURTO_RAPIÑA_Montevideo.html'.")